In [1]:
# Importing the necessary libraries for Data preparation, EDA and prediction models:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns                       #visualisation
import matplotlib.pyplot as plt             #visualisation
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.ensemble import RandomForestClassifier
%matplotlib inline     
sns.set(color_codes=True)


# Importing the dataset - saved as a CSV file
df = pd.read_csv("/kaggle/input/data-science-salaries-2023/ds_salaries.csv")
# To display the top 10 rows 
df.head(10)  

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,USD,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L
7,2023,SE,FT,Data Scientist,219000,USD,219000,CA,0,CA,M
8,2023,SE,FT,Data Scientist,141000,USD,141000,CA,0,CA,M
9,2023,SE,FT,Data Scientist,147100,USD,147100,US,0,US,M


In [2]:
#Creating a data dictionary for better understanding of the dataset
df.dtypes

work_year              int64
experience_level      object
employment_type       object
job_title             object
salary                 int64
salary_currency       object
salary_in_usd          int64
employee_residence    object
remote_ratio           int64
company_location      object
company_size          object
dtype: object

**Data Dictionary:**
* work_year - date variable (year)
* experience_level - Categorial Variable (experience level of each employee - 4 categories)
* employment_type - Categorial Variable (showing each type of employment - 3 categories)
* job_title - String variable (All the job positions presented in the dataset)
* salary - Currency Variable (Salary for each employee)
* salary_currency - String Variable (currency for each salary)
* salary_in_usd - Currency Variabele - Varible salary converted to the USD currency
* employee_residence - String Variable (Locations for the different employee residences)
* company_location - String Variable (Locations for all the offices)
* remote_ratio - Categorical variable with 3 categories which hold information for how remote their work is
* Company_size - Categorical variable.

**Note:**
Variables Company_size and remote_ratio were encoded in Power BI for better understanding of the data.
Company_size --) company_size_numbers (new variable) with 3 categories: Startup,100-999 employees and More than 1000 employees.
remote_ratio with 3 categories: 0,50 and 100 --) Telework (new variable) with 3 categories:On-site,Hybrid and Fully Remote.

In [3]:
# Changing the name of the variables
df = df.rename(columns={"work_year": "Date(Year)"})
df = df.rename(columns={"employee_residence": "residence_location"})
# To display the top 10 rows 
df.head(10) 

,Date(Year),experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,residence_location,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M
5,2023,SE,FT,Applied Scientist,222200,USD,222200,US,0,US,L
6,2023,SE,FT,Applied Scientist,136000,USD,136000,US,0,US,L
7,2023,SE,FT,Data Scientist,219000,USD,219000,CA,0,CA,M
8,2023,SE,FT,Data Scientist,141000,USD,141000,CA,0,CA,M
9,2023,SE,FT,Data Scientist,147100,USD,147100,US,0,US,M


In [4]:
# Furthermore, there is no need to investigate the duplicates in the dataset - as the dataset has location values - it is expected to have duplicated values.
# Nevertheless, the code below finds any duplicates within the current dataset. 

df.shape #  returns the shape of the array
duplicate_rows_df = df[df.duplicated()]
print("Number of duplicate rows in the current dataset: ", duplicate_rows_df.shape)

Number of duplicate rows in the current dataset:  (1171, 11)


In [5]:
print(df.isnull().sum())
# Variable age has 263 missing values. 
df.describe() # returns the descriptive statistics of each variable, including the mean (30 years old)

Date(Year)            0
experience_level      0
employment_type       0
job_title             0
salary                0
salary_currency       0
salary_in_usd         0
residence_location    0
remote_ratio          0
company_location      0
company_size          0
dtype: int64


,Date(Year),salary,salary_in_usd,remote_ratio
count,3755.000000,3.755000e+03,3755.000000,3755.000000
mean,2022.373635,1.906956e+05,137570.389880,46.271638
std,0.691448,6.716765e+05,63055.625278,48.589050
min,2020.000000,6.000000e+03,5132.000000,0.000000
25%,2022.000000,1.000000e+05,95000.000000,0.000000
50%,2022.000000,1.380000e+05,135000.000000,0.000000
75%,2023.000000,1.800000e+05,175000.000000,100.000000
max,2023.000000,3.040000e+07,450000.000000,100.000000


In [6]:
from IPython.display import IFrame

embed_url = "https://app.powerbi.com/reportEmbed?reportId=9019de11-58ed-4011-9c1b-b0ffbf4a45b6&autoAuth=true&ctid=ae3f47d1-dd89-43b1-a9b5-f7aadd7b81af"

IFrame(embed_url, width=800, height=600)
